In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [153]:
# Importing library
import numpy as np
import pandas as pd

# BeautifulSoup = hapus tag html
from bs4 import BeautifulSoup 
import re # regular expressions (regex)

# natural language tool kits
from nltk.corpus import stopwords
import nltk

# word2vec library
from gensim.models import word2vec
import itertools

# Packages required for data preparation
from sklearn.model_selection import train_test_split

# library untuk Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# # path dataset
# data_path = r'/content/drive/MyDrive/Text Dataset/hate_speech.csv'
# dataset = pd.read_csv(data_path, header=0, sep = ',')

# # dataset = dataset.reindex(np.random.permutation(dataset.index))  

In [182]:
# path dataset
data_path = r'/content/drive/MyDrive/Text Dataset/sosmed-disaster.csv'
dataset = pd.read_csv(data_path, header=0, encoding='latin-1')
dataset.head(10)
# dataset = dataset.reindex(np.random.permutation(dataset.index))  

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one,choose_one:confidence,choose_one_gold,keyword,location,text,tweetid,userid
0,778243823,True,golden,156,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just happened a terrible car crash,1.0,NaN
1,778243824,True,golden,152,NaN,Relevant,1.0000,Relevant,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,13.0,NaN
2,778243825,True,golden,137,NaN,Relevant,1.0000,Relevant,NaN,NaN,"Heard about #earthquake is different cities, s...",14.0,NaN
3,778243826,True,golden,136,NaN,Relevant,0.9603,Relevant,NaN,NaN,"there is a forest fire at spot pond, geese are...",15.0,NaN
4,778243827,True,golden,138,NaN,Relevant,1.0000,Relevant,NaN,NaN,Forest fire near La Ronge Sask. Canada,16.0,NaN
5,778243828,True,golden,140,NaN,Relevant,1.0000,Relevant,NaN,NaN,All residents asked to 'shelter in place' are ...,17.0,NaN
6,778243831,True,golden,142,NaN,Relevant,1.0000,Relevant,NaN,NaN,"13,000 people receive #wildfires evacuation or...",18.0,NaN
7,778243832,True,golden,151,NaN,Relevant,1.0000,Relevant,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,19.0,NaN
8,778243833,True,golden,143,NaN,Relevant,1.0000,Relevant,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,20.0,NaN
9,778243834,True,golden,136,NaN,Relevant,0.9606,Relevant\nCan't Decide,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,21.0,NaN


In [172]:
dataset = dataset[['text', 'choose_one']]
# # dataset.text = dataset.text.apply(remove_stopwords).apply(remove_mentions)

dataset.head(10)

,text,choose_one
0,Just happened a terrible car crash,Relevant
1,Our Deeds are the Reason of this #earthquake M...,Relevant
2,"Heard about #earthquake is different cities, s...",Relevant
3,"there is a forest fire at spot pond, geese are...",Relevant
4,Forest fire near La Ronge Sask. Canada,Relevant
5,All residents asked to 'shelter in place' are ...,Relevant
6,"13,000 people receive #wildfires evacuation or...",Relevant
7,Just got sent this photo from Ruby #Alaska as ...,Relevant
8,#RockyFire Update => California Hwy. 20 closed...,Relevant
9,Apocalypse lighting. #Spokane #wildfires,Relevant


In [156]:
dataset.dropna(how='any')

,text,choose_one
0,Just happened a terrible car crash,Relevant
1,Our Deeds are the Reason of this #earthquake M...,Relevant
2,"Heard about #earthquake is different cities, s...",Relevant
3,"there is a forest fire at spot pond, geese are...",Relevant
4,Forest fire near La Ronge Sask. Canada,Relevant
...,...,...
10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,Relevant
10872,Police investigating after an e-bike collided ...,Relevant
10873,The Latest: More Homes Razed by Northern Calif...,Relevant
10874,MEG issues Hazardous Weather Outlook (HWO) htt...,Relevant


In [183]:
dataset.isnull().any()

_unit_id                 False
_golden                  False
_unit_state              False
_trusted_judgments       False
_last_judgment_at         True
choose_one               False
choose_one:confidence    False
choose_one_gold           True
keyword                   True
location                  True
text                     False
tweetid                  False
userid                    True
dtype: bool

In [184]:
# def remove_pattern(input_txt, pattern):
#     r = re.findall(pattern, input_txt)
#     for i in r:
#         input_txt = re.sub(i, '', input_txt)
        
#     return input_txt

# # remove twitter handles (@user)
# dataset['tidy_tweet'] = np.vectorize(remove_pattern)(dataset['text'], "@[\w]*")

# remove special characters, numbers, punctuations
dataset['tidy_tweet'] = dataset['text'].str.replace("[^a-zA-Z#]", " ")

# remove short word
dataset['tidy_tweet'] = dataset['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [127]:
dataset.isnull().any()

text          False
choose_one    False
tidy_tweet    False
dtype: bool

In [185]:
X_train, X_test, y_train, y_test = train_test_split(dataset['tidy_tweet'], dataset['choose_one'], test_size = 0.2, random_state = 42)

In [129]:
# proprocessing data teks
def review_wordlist(review, remove_stopwords=False):
    
    # # hapus simbol
    # review_text = re.sub("[^a-zA-Z]"," ",review)
    
    # # hapus @
    # review_text = re.sub("([^\s\w]|_@!?)+"," ", review_text)

    # konversi ke huruf kecil dan dipisah perkata
    words = review.lower().split()

    # menghapus stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
    
    return(words)

In [93]:
# download file punctuation and stpwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
# word2vec expects a list of lists.
# Using punkt tokenizer for better splitting of a paragraph into sentences.

import nltk.data
#nltk.download('popular')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [95]:
# Memisah review per kalimat
def review_sentences(review, tokenizer, remove_stopwords=False):
    
    # melakukan tokenize dengan nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []

    # mengisi array sentences dengan masing - masing review
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence, remove_stopwords))

    # list dari list
    return sentences

In [177]:
sentences = []
print("Parsing sentences from training set")
for review in X_train:
    sentences += review_sentences(review, tokenizer)

Parsing sentences from training set


In [178]:
print("List dari lists. Cek tipe data : ", type(sentences), " of ", type(sentences[0]))
# print(sentences)
print(sentences[0:10])
print(len(sentences))

List dari lists. Cek tipe data :  <class 'list'>  of  <class 'list'>
[['dont', 'even', 'remember', 'slsp', 'happening', 'just', 'remember', 'being', 'like', 'then', 'lights', 'turned', 'everyone', 'screamed', 'encore'], ['hazelannmac', 'feel', 'guilty', 'about', 'wishing', 'hatman', 'mudslide', 'delicious'], ['soultech', 'collide', 'club', 'http', 'xixbspot'], ['police', 'officer', 'wounded', 'suspect', 'dead', 'after', 'exchanging', 'shots', 'http', 'iphazv'], ['cramer', 'iger', 'words', 'that', 'wrecked', 'disney', 'stock', 'http', 'dgpbaivl'], ['safyuan', 'just', 'minor', 'citation', 'possesion', 'decriminalized', 'substance', 'facing', 'time'], ['thesmallclark', 'kill', 'instead', 'survived', 'shot', 'exactly', 'know', 'fled', 'scene', 'pulled', 'trigger', 'with'], ['could', 'demolish', 'this', 'right', 'https', 'jccrj'], ['national', 'park', 'services', 'tonto', 'national', 'forest', 'stop', 'annihilation', 'salt', 'river', 'wild', 'horse', 'https', 'fekgyby', 'change'], ['ashnigg

In [98]:
best_score = []
best_parameter = []

In [186]:
# =================================================================================
# word2vec mulai disini
# =================================================================================

# word2vec model
model = word2vec.Word2Vec(workers = 4, 
                      size = 200, 
                      min_count = 10, 
                      window = 10)
model.build_vocab(sentences)

# training model word2vec (CBOW, karena parameter 'sg' menggunakan nilai default)
print('\n==============================================================')
print("Training model Word2Vec ...")
print('==============================================================')
model.train(sentences = sentences, total_examples = len(sentences), epochs = 5)

# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(200,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),200),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, 200)
        counter = counter+1
        
    return reviewFeatureVecs

# Calculating average feature vector (mendapatkan vektor training set)
clean_train_reviews = []
for review in X_train:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, 200)

# Calculating average feature vactors (mendapatkan vektor test set)     
clean_test_reviews = []
for review in X_test:
    clean_test_reviews.append(review_wordlist(review,remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, 200)

# tujuan masing-masing model word2vec, untuk mendapatkan 'trainDataVecs' dan 'testDataVecs' 
# untuk diolah lebih lanjut di classifier

# =================================================================================
# word2vec berakhir disini
# =================================================================================


Training model Word2Vec ...
Review 0 of 8700
Review 1000 of 8700
Review 2000 of 8700
Review 3000 of 8700

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide



Review 4000 of 8700
Review 5000 of 8700
Review 6000 of 8700
Review 7000 of 8700
Review 8000 of 8700
Review 0 of 2176
Review 1000 of 2176
Review 2000 of 2176


In [139]:
model.doesnt_match(['car','pregnant','president','wild','nigguh'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'pregnant'

In [180]:
print(trainDataVecs.shape)
print(type(trainDataVecs[10][10]))
print(trainDataVecs[10])

(8700, 200)
<class 'numpy.float32'>
[-0.10271084 -0.03170505 -0.00098252 -0.18023546 -0.0228553   0.04365844
  0.05564448  0.04348141  0.01742684  0.03555949  0.06648624  0.16923879
  0.22512035 -0.01368562  0.10836975  0.01556964 -0.0257571   0.30390897
  0.13858673 -0.10075767  0.01463384 -0.0031422   0.05139892  0.00566258
 -0.08529083 -0.1414064   0.06621731 -0.14449589  0.2560714  -0.00662073
 -0.1119406  -0.17266543 -0.0371987  -0.10893557 -0.20561504 -0.13479587
 -0.00904985  0.17705944  0.03107837  0.2551353   0.04833467  0.13289365
  0.38831037  0.00733981 -0.09739133 -0.22222874  0.11374351 -0.09841061
 -0.14845452  0.08782728 -0.02101293  0.25709835 -0.0963615   0.02582991
 -0.10871396 -0.08586558  0.13421634 -0.18128519  0.07976219  0.09458257
  0.06633149 -0.16116491 -0.11419448  0.04752856  0.1435065  -0.12124445
 -0.06443848 -0.11143015  0.10988839 -0.00739202 -0.14813554  0.03208847
 -0.09342368  0.18080726  0.01476715 -0.24182959  0.14024012  0.02547339
  0.0873663   0

In [120]:
# np.isfinite(trainDataVecs)
# np.isnan(trainDataVecs)
print('\n')
# np.where(np.isnan(trainDataVecs))
np.nan_to_num(trainDataVecs)

array([[-0.11027239, -0.0527853 ,  0.01780591, ...,  0.03072819,
        -0.09230059, -0.04021281],
       [-0.11575003, -0.05665668,  0.02204461, ...,  0.03352845,
        -0.09540892, -0.04044687],
       [-0.13312593, -0.06721942,  0.01607215, ...,  0.04303947,
        -0.11139769, -0.04966934],
       ...,
       [-0.13963467, -0.06781787,  0.02091169, ...,  0.04092883,
        -0.11604854, -0.05155595],
       [-0.11602937, -0.05895108,  0.01659537, ...,  0.03518617,
        -0.09946667, -0.04283651],
       [-0.09914498, -0.04959012,  0.0138425 , ...,  0.03421538,
        -0.08551119, -0.03520462]], dtype=float32)

In [103]:
np.nan_to_num(testDataVecs)

array([[-0.1110994 , -0.08648997,  0.01211994, ..., -0.00258085,
        -0.11481857, -0.04226538],
       [-0.17175099, -0.13346897,  0.01741779, ...,  0.00241178,
        -0.18210906, -0.06156302],
       [-0.11974993, -0.09258076,  0.01487163, ..., -0.00138354,
        -0.12524632, -0.04453228],
       ...,
       [-0.11395289, -0.08568748,  0.01341498, ..., -0.00245851,
        -0.11738835, -0.04216096],
       [-0.16710521, -0.12803331,  0.01367831, ..., -0.00448012,
        -0.18194926, -0.06190185],
       [-0.11338797, -0.09068947,  0.0116854 , ..., -0.00114231,
        -0.12670566, -0.04285615]], dtype=float32)

In [187]:
np.any(np.isinf(trainDataVecs))

False

In [163]:
from sklearn.preprocessing import StandardScaler

# normalisasi
# atr_train = StandardScaler().fit_transform(atr_train)
# sc = StandardScaler()

trainDataVecs = StandardScaler().fit_transform(trainDataVecs)
testDataVecs = sc.fit(testDataVecs)

In [188]:
print('\n==============================================================')
print('Sentiment Analysis Process ...')
print('==============================================================')
# Memanggil classifier
score, report = classification_model(trainDataVecs, testDataVecs, y_train, y_test)

print(f'Score : {score}')


Sentiment Analysis Process ...


ValueError: ignored

**SEACRH**

In [ ]:
# parameter yang akan dioptimasi
parm_dict = {
    'workers' : (4,),
    'size' : (300, 350, 400, 450, 500, 550, 600),
    'min_count' : (40,),
    'window' : (10,)
}

# melakukan x, dengan metode y, untuk z 
# optimization of feature dimension on word embedding based on vector size using grid search for sentimen analysis

def cust_param_search(parm_dict):

    score_best, parm_best = 0,()
    workers, size, min_count, window = [tup for k,tup in parm_dict.items()] # Individual parm tuples
    parm_combo = list(itertools.product(workers, size, min_count, window)) # Create all combinations
    
    print('\n==============================================================')
    print('PARAMETER')
    print('==============================================================')
    print(f'Total kombinasi parameter   : {len(parm_combo)}')
    print(f'Semua kombinasi parameter   : {parm_combo}')

    # Grid search
    i = 1
    for parms in parm_combo:

        print('\n==============================================================')
        print(f'{i}.\t Parameter : {parms}')
        print('==============================================================')
        
        w, s, m, wi = parms
        
        # =================================================================================
        # word2vec mulai disini
        # =================================================================================
        
        # word2vec model
        model = word2vec.Word2Vec(workers = w, 
                              size = s, 
                              min_count = m, 
                              window = wi)
        model.build_vocab(sentences)

        # training model word2vec (CBOW, karena parameter 'sg' menggunakan nilai default)
        print('\n==============================================================')
        print("Training model Word2Vec ...")
        print('==============================================================')
        model.train(sentences = sentences, total_examples = len(sentences), epochs = 5)

        # Function to average all word vectors in a paragraph
        def featureVecMethod(words, model, num_features):
            # Pre-initialising empty numpy array for speed
            featureVec = np.zeros(s,dtype="float32")
            nwords = 0
            
            #Converting Index2Word which is a list to a set for better speed in the execution.
            index2word_set = set(model.wv.index2word)
            
            for word in  words:
                if word in index2word_set:
                    nwords = nwords + 1
                    featureVec = np.add(featureVec,model[word])
            
            # Dividing the result by number of words to get average
            featureVec = np.divide(featureVec, nwords)
            return featureVec

        # Function for calculating the average feature vector
        def getAvgFeatureVecs(reviews, model, num_features):
            counter = 0
            reviewFeatureVecs = np.zeros((len(reviews),s),dtype="float32")
            for review in reviews:
                # Printing a status message every 1000th review
                if counter%1000 == 0:
                    print("Review %d of %d"%(counter,len(reviews)))
                    
                reviewFeatureVecs[counter] = featureVecMethod(review, model, s)
                counter = counter+1
                
            return reviewFeatureVecs

        # Calculating average feature vector (mendapatkan vektor training set)
        clean_train_reviews = []
        for review in X_train:
            clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
            
        trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, s)

        # Calculating average feature vactors (mendapatkan vektor test set)     
        clean_test_reviews = []
        for review in X_test:
            clean_test_reviews.append(review_wordlist(review,remove_stopwords=True))
            
        testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, s)

        # tujuan masing-masing model word2vec, untuk mendapatkan 'trainDataVecs' dan 'testDataVecs' 
        # untuk diolah lebih lanjut di classifier
        
        # =================================================================================
        # word2vec berakhir disini
        # =================================================================================

        print('\n==============================================================')
        print('Sentiment Analysis Process ...')
        print('==============================================================')
        # Memanggil classifier
        score, report = classification_model(trainDataVecs, testDataVecs, y_train, y_test)

        # temporary print
        print('\n==============================================================')
        print(f'Hasil Test Parameter ke - {i}')
        print(f'Parameter  => {parms}')
        print(f'Accuracy   => {score}')
        print(f'Classification Report => \n {report}')

        best_score.append(score)
        best_parameter.append(parms)
        
        if score > score_best:
            score_best = score
            parm_best = parms
        
        i = i + 1
    
    print('\n==============================================================')
    print(f'Best Parameter  => {parm_best}')
    print(f'Accuracy   => {score_best}')
    print('==============================================================')

In [ ]:
def classification_model(train_feature_vec, test_feature_vec, train_sentiment, test_sentiment):

    # Model/classifier yang dipakai bebas
    # Fit random forest classifier ke data training
    forest = RandomForestClassifier(n_estimators = 100)    
    forest = forest.fit(train_feature_vec, train_sentiment) 

    # Prediksi nilai sentiment untuk test data 
    predicted = forest.predict(test_feature_vec)

    # akurasi
    accuracy = accuracy_score(test_sentiment, predicted)
    report = classification_report(test_sentiment, predicted, digits = 5)
    
    return accuracy, report

In [ ]:
# test drive ma men (estimasi running : 21.30)
cust_param_search(parm_dict)


PARAMETER
Total kombinasi parameter   : 7
Semua kombinasi parameter   : [(4, 300, 40, 10), (4, 350, 40, 10), (4, 400, 40, 10), (4, 450, 40, 10), (4, 500, 40, 10), (4, 550, 40, 10), (4, 600, 40, 10)]

1.	 Parameter : (4, 300, 40, 10)

Training model Word2Vec ...
Review 0 of 19826
Review 1000 of 19826
Review 2000 of 19826
Review 3000 of 19826
Review 4000 of 19826
Review 5000 of 19826


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in true_divide


Review 6000 of 19826
Review 7000 of 19826
Review 8000 of 19826
Review 9000 of 19826
Review 10000 of 19826
Review 11000 of 19826
Review 12000 of 19826
Review 13000 of 19826
Review 14000 of 19826
Review 15000 of 19826
Review 16000 of 19826
Review 17000 of 19826
Review 18000 of 19826
Review 19000 of 19826
Review 0 of 4957
Review 1000 of 4957
Review 2000 of 4957
Review 3000 of 4957
Review 4000 of 4957

Sentiment Analysis Process ...


ValueError: ignored

In [ ]:
print(best_score)
print(best_parameter)